# Setting Wind speed thresholds

This notebook looks at the wind speed forecasts just before and just after landfall to get an indication of what the max wind speed would be for storms and sets a threshold.

In [ ]:
%load_ext jupyter_black

import os
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.express as px
import glob
import warnings
from src.constants import *
from src.utils import *
from src.datasources import rsmc, helpers

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [ ]:
warnings.filterwarnings("ignore")
save_dir = Path(AA_DATA_DIR) / "public" / "exploration" / "moz" / "ecmwf_hindcast"
adm1_path = (
    Path(AA_DATA_DIR)
    / "public"
    / "raw"
    / "moz"
    / "cod_ab"
    / "moz_admbnda_adm1_ine_20190607.shp"
)
gdf_adm1 = gpd.read_file(adm1_path)
gdf_sel = gdf_adm1[gdf_adm1.ADM1_PT.isin(ADMS)]

In [5]:
moz_cyclones = [
    Path(f).stem.replace("_all", "").upper()
    for f in glob.glob(str(Path(save_dir) / "csv" / "*_all.csv"))
]

In [ ]:
cyclone_speed = []
for cyclone_file_path in glob.glob(str(save_dir / "csv/*_all.csv")):
    cyclone_name = Path(cyclone_file_path).stem.split("_")[0]
    print(f"Processing file: {cyclone_file_path}")
    cyclone_file = pd.read_csv(cyclone_file_path)
    cyclone_file["time"] = pd.to_datetime(cyclone_file["time"])

    cyclone_df = (
        cyclone_file[["time", "speed", "lat", "lon", "lead_time", "forecast_time"]]
        .groupby(["time", "forecast_time"])
        .median()
        .reset_index()
    )
    cyclone_df["lat"] = cyclone_df["lat"].apply(lambda x: -x if x > 0 else x)

    cyclone_df["speed_knots"] = cyclone_df["speed"] * 1.94384
    cyclone_df["storm_category"] = cyclone_df["speed_knots"].apply(categorize_cyclone)
    cyc_ls = []
    lt_0_df = gpd.GeoDataFrame(
        cyclone_df,
        geometry=gpd.points_from_xy(cyclone_df.lon, cyclone_df.lat),
        crs="EPSG:4326",
    )
    cyc_sjoin = gpd.sjoin(lt_0_df, gdf_sel, how="left", predicate="intersects")
    lt_0_df["within_land"] = cyc_sjoin["index_right"].notna()
    lt_0_df["point_location"] = np.where(lt_0_df["within_land"], "Within", "Outside")

    first_landfall = (
        lt_0_df[lt_0_df["within_land"]].index[0]
        if not lt_0_df[lt_0_df["within_land"]].empty
        else None
    )
    if first_landfall is None:
        continue
    lt_0_df.loc[lt_0_df.index == first_landfall, "point_location"] = "Landfall"
    landfall_time = pd.to_datetime(
        lt_0_df[lt_0_df["point_location"] == "Landfall"]["time"].values[0]
    )
    lt_0_df["time_to_landfall"] = (
        landfall_time - pd.to_datetime(lt_0_df["forecast_time"])
    ).dt.total_seconds() / 3600
    lt_0_df["time_from_landfall"] = lt_0_df["lead_time"] - lt_0_df["time_to_landfall"]
    # lt_0_df = lt_0_df[(lt_0_df["time_to_landfall"] <= 12)]
    cyc_df = lt_0_df  # [(lt_0_df["point_location"] != "Outside") | ((lt_0_df["time_from_landfall"] >= -12) & (lt_0_df["time_from_landfall"] <= 12))]
    cyc_ls.append(cyc_df)
    if len(cyc_ls) > 0:
        cyc_df = pd.concat(cyc_ls)
        cyc_df["storm"] = cyclone_name.upper()
        cyclone_speed.append(cyc_df)

cyclone_out = pd.concat(cyclone_speed)

In [13]:
storm_max_ws = (
    cyclone_out[
        (cyclone_out["time_to_landfall"].between(-12, 12, inclusive="both"))
        & cyclone_out["within_land"]
    ]
    .groupby("storm")["speed_knots"]
    .max()
    .reset_index()
)

In [24]:
storm_max_ws[storm_max_ws["speed_knots"] >= 48]

,storm,speed_knots
4,DINEO,56.954512
5,ELOISE,56.954512
7,FILIPO,48.012848
8,FREDDY,55.010672
10,IDAI,68.034400


In [ ]:
# Sort the values by speed_knots in ascending order
df_sorted = storm_max_ws.sort_values(by="speed_knots", ascending=True)

fig = px.bar(
    df_sorted,
    x="storm",
    y="speed_knots",
    title="Maximum Storm Speed on Land in Knots",
    labels={"storm": "Storm", "speed_knots": "Speed (knots)"},
)
fig.update_layout(xaxis={"categoryorder": "total ascending", "tickangle": -40})

fig.show()